In [1]:
import pandas as pd
import numpy as np
import requests
import json
from collections import Counter

In [2]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [3]:
def CountList(e):
    return Counter(e)
def Uniquefy(e):
    return list(set(e))

In [4]:
# Leitura do dataset
champ = pd.read_csv('participants.csv')
champ = champ.loc[champ['gameDuration'] > 1500]
champ['played'] = 1

/home/neto/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (55,56) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [5]:
gpby = champ.groupby(['participantId'])['championId'].apply(list).reset_index(name='championsPlayed')
gpby['count'] = gpby['championsPlayed'].apply(len)
gpby = gpby.loc[gpby['count'] > 5]

In [6]:
df_usable = gpby.explode('championsPlayed')
df_usable['played'] = 1
df_usable.drop('count',axis='columns', inplace=True)
df_usable

,participantId,championsPlayed,played
0,--Ju_A1b8ZlsnI5b8qovR9jFOOtJ4Ae1xmfIeX0V_ZBjqC...,14,1
0,--Ju_A1b8ZlsnI5b8qovR9jFOOtJ4Ae1xmfIeX0V_ZBjqC...,84,1
0,--Ju_A1b8ZlsnI5b8qovR9jFOOtJ4Ae1xmfIeX0V_ZBjqC...,62,1
0,--Ju_A1b8ZlsnI5b8qovR9jFOOtJ4Ae1xmfIeX0V_ZBjqC...,164,1
0,--Ju_A1b8ZlsnI5b8qovR9jFOOtJ4Ae1xmfIeX0V_ZBjqC...,106,1
...,...,...,...
8096,zzy0pGrzLeoBMCdUwni5AGJ520w0rGROfBCThrSkViN85RA,3,1
8096,zzy0pGrzLeoBMCdUwni5AGJ520w0rGROfBCThrSkViN85RA,147,1
8096,zzy0pGrzLeoBMCdUwni5AGJ520w0rGROfBCThrSkViN85RA,12,1
8096,zzy0pGrzLeoBMCdUwni5AGJ520w0rGROfBCThrSkViN85RA,12,1


In [7]:
df_grouped = df_usable.groupby(['participantId', 'championsPlayed'])['played'].sum()
cesta = df_grouped.unstack(fill_value=0)
cesta.reset_index().set_index('participantId')
df_grouped

participantId                                             championsPlayed
--Ju_A1b8ZlsnI5b8qovR9jFOOtJ4Ae1xmfIeX0V_ZBjqCsNQ3IshLwd  6                  4
                                                          14                 2
                                                          24                 2
                                                          31                 2
                                                          39                 3
                                                                            ..
zzy0pGrzLeoBMCdUwni5AGJ520w0rGROfBCThrSkViN85RA           117                1
                                                          147                5
                                                          201                1
                                                          235                2
                                                          526                3
Name: played, Length: 14118, dtype: int64

In [8]:
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

cesta = cesta.applymap(encode_units)
frequent_itemset = apriori(cesta, min_support=0.01, use_colnames=True)

In [18]:
recommendList = []
for i in [12, 111, 497]:
    def compare (a):
        return a.__contains__(i)
    rules = association_rules(frequent_itemset,metric="confidence",min_threshold=0.01)
    rules_test = rules[rules['antecedents'].map(compare)]
    lists = rules_test.sort_values('support', ascending=False).head(5)['consequents'].tolist()
    mapItemset = map(list, lists)
    listItems = list(mapItemset)
    newList = []
    for l in listItems:
        newList.extend(l)
    recommendList.extend(newList)
recommendList = [s for s in recommendList if s not in [12, 111, 497]]
recommendCount = CountList(recommendList)

In [19]:
recommendCount

Counter({412: 3, 43: 3, 89: 3, 235: 2, 526: 1})

In [25]:
recommendCount = dict(sorted(recommendCount.items(), key=lambda item: item[1], reverse=True))

In [32]:
list(recommendCount.keys())[0:5]

[412, 43, 89]